In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf # use it to train the module
from sklearn.preprocessing import LabelEncoder # encoder to make the dataset as number
from sklearn.preprocessing import OneHotEncoder # encoder to make the dataset as one hot
from sklearn.utils import shuffle # use for shuffle the data
from sklearn.model_selection import train_test_split # split the data set
from sklearn.preprocessing import normalize # use to normalize the data set


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# get the training se
df_origin = pd.read_csv("../input/train.csv")
train_df = df_origin.copy()
train_df.head()

['.DS_Store', 'test.csv', 'train.csv', 'gender_submission.csv']


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [41]:
# # split the data set to X_train and Y_train
# Y_train = train_df['Survived']
# X_train = train_df.drop(['Survived'], axis=1)
# print('X_train: ', X_train.shape)
# print('Y_train: ', X_train.shape)
# print('-------------')
# X_train.head()

In [42]:
print(train_df.isnull().any())

# get the data distribution
train_df.describe()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


# Set the sex to binary value

In [43]:
print(train_df['Sex'].describe())
#print(np.bincount(X_train['Sex']))

# set the sex to binary value
# le_sex = LabelEncoder().fit(train_df['Sex'])
# Sex_label = le_sex.transform(train_df['Sex']) # don't give the result to dataframe directely, we need to one hot encode later
# # Set the sex to on-hot
# #    545                     "Reshape your data either using array.reshape(-1, 1) if "
# #     546                     "your data has a single feature or array.reshape(1, -1) "
# # --> 547                     "if it contains a single sample.".format(array))
# # categories='auto' is follow the warning instruction
# OH_sex = OneHotEncoder(sparse=False, categories='auto')
# Sex_label = OH_sex.fit_transform(Sex_label.reshape(-1, 1 ))

# print(Sex_label)
# #train_df['Sex_Lable'] = Sex_label
# train_df['Sex_Lable'] = np.NaN
# train_df['Sex_Lable'] = Sex_label

# print(train_df['Sex_Lable'].iloc[0])
# print(train_df['Sex_Lable'].iloc[0].dtype)
############ old method #########################
# X_train['Sex_binary'] = X_train['Sex'] == 'male'
# #X_train['Sex_binary'].head()
# X_train['Sex_binary'] = X_train['Sex_binary'].astype('int')
# #X_train['Sex'].head()

# print(X_train['Sex_binary'].describe())
# # check the covariance
# print(np.cov(X_train['Sex_binary'], train_df.Survived))
sex_df = pd.get_dummies(df_origin['Sex'], columns='Sex', prefix='Sex')
train_df = pd.concat([train_df, sex_df], axis=1, sort=False)
train_df = train_df.drop(['Sex'], axis=1)
#train_df.head()

count      891
unique       2
top       male
freq       577
Name: Sex, dtype: object


In [44]:
# feature engineering on Embarked

# deal with the NaN or just use the NaN as a feature
# print(df_origin['Embarked'].isnull().any())
# Embarked_nan_mask = df_origin['Embarked'].isnull()
# #Try using .loc[row_indexer,col_indexer] = value instead
# Embarked_lable = train_df['Embarked'].copy()
# Embarked_lable[Embarked_nan_mask] = 'N'
# train_df['Embarked_lable'] = Embarked_lable
# print(train_df['Embarked_lable'].isnull().any())
# print(train_df[Embarked_nan_mask])
eb_df = pd.get_dummies(df_origin['Embarked'], columns='Embarked', dummy_na=True, prefix='Embarked')
train_df = pd.concat([train_df, eb_df], axis=1, sort=False)
train_df = train_df.drop(['Embarked'], axis=1)


#one hot encode
# le_Eb = LabelEncoder().fit(train_df['Embarked_lable'])
# train_df['Embarked_lable'] = le_Eb.transform(train_df['Embarked_lable']) # don't give the result to dataframe directely, we need to one hot encode later
# #print(train_df['Embarked_lable'])

# # categories='auto' is follow the warning instruction
# #  sklearn 的新版本中，OneHotEncoder 的输入必须是 2-D array，而 testdata.age 返回的 Series 本质上是 1-D array，所以要改成
# # 
# a = OneHotEncoder(sparse=False, categories='auto').fit_transform(train_df[['Embarked_lable']])
# print(a)


#print(train_df['Embarked_lable'])
#train_df['Sex_Lable'] = Sex_label
#train_df['Embarked_lable'] = Eb_label


In [45]:
# deal with the Cabin
print(df_origin['Cabin'].describe())
print(df_origin['Cabin'][0], 'is ', type(df_origin['Cabin'][0]))
print(df_origin['Cabin'][1], 'is ', type(df_origin['Cabin'][1]))

# the nan in cabin type is float
# cabin_nan_mask = df_origin['Cabin'].isna()
# print(cabin_nan_mask)
# df_origin[cabin_nan_mask]

def get_cabin_level(x):
    #print(x)
    if x == 0:
        return x
    else:
        return x[0]

# make the NaN to 0
train_df['Cabin_label']= train_df['Cabin'].fillna(0)
    
# # modify to cabin level, such as C23 to level C
train_df['Cabin_label'] = train_df['Cabin_label'].map(get_cabin_level)
#train_df = train_df.drop(['Cabin'], axis=1)

cb_df = pd.get_dummies(train_df['Cabin_label'], columns='Cabin_label', dummy_na=True, prefix='Cabin')
train_df = pd.concat([train_df, cb_df], axis=1, sort=False)
train_df = train_df.drop(['Cabin_label'], axis=1)  
train_df = train_df.drop(['Cabin'], axis=1)

count             204
unique            147
top       C23 C25 C27
freq                4
Name: Cabin, dtype: object
nan is  <class 'float'>
C85 is  <class 'str'>


In [46]:
# deal with the age
train_df['Age_label']= np.floor(train_df['Age'].fillna(np.mean(df_origin['Age'])))
train_df = train_df.drop(['Age'], axis=1)   

In [47]:
# drop unneed column
train_df = train_df.drop(['Name', 'Ticket', 'PassengerId'], axis=1)

# shuffle the data
train_df = shuffle(train_df)

In [48]:
# normalize the fare
f_temp = train_df['Fare'].fillna(np.mean(df_origin['Fare']))
fare_max = np.max(f_temp)
fare_min = np.min(f_temp)
train_df['Fare'] = f_temp.apply(lambda x: (x - fare_min) / (fare_max - fare_min))

In [49]:
Y = train_df['Survived'].values
X = train_df.drop(['Survived'], axis=1).values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
m_feature = X_train.shape[1]
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 712
number of test examples = 179
X_train shape: (712, 21)
Y_train shape: (712,)
X_test shape: (179, 21)
Y_test shape: (179,)


In [54]:
def add_layer(inputs, in_size, out_size, activation_function = None):
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
        
    return outputs

xs = tf.placeholder(tf.float32,  [None, m_feature])
ys = tf.placeholder(tf.int32,  [None])
# add relu here
#l1 = add_layer(xs, m_feature, 5, activation_function=tf.nn.sigmoid)
logits = add_layer(xs, m_feature, 2, activation_function=tf.nn.sigmoid)
predict = tf.arg_max(logits, 1)

loss = tf.losses.sparse_softmax_cross_entropy(logits=logits,labels=ys)
loss = tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)

_, acc_op = tf.metrics.accuracy(labels=ys,predictions=predict)

In [55]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())

    for epoch in range(10000): # 训练次数
        loss_value,_ ,acc_value= sess.run([loss,optimizer,acc_op],feed_dict={xs:X_train,ys:Y_train})
        if epoch%1000 == 0:
            print('loss = {}, acc = {}'.format(loss_value,acc_value))
    acc_value = sess.run([acc_op],feed_dict={xs:X_test,ys:Y_test}) # 验证集的精度，这里验证集和
    print('val acc = {}'.format(acc_value))


loss = 0.6932179927825928, acc = 0.6348314881324768
loss = 0.48382022976875305, acc = 0.7868171334266663
loss = 0.4761705696582794, acc = 0.8052294254302979
loss = 0.47204533219337463, acc = 0.8132410645484924
loss = 0.46928173303604126, acc = 0.8171696662902832
loss = 0.4671750068664551, acc = 0.8181925415992737
loss = 0.46563708782196045, acc = 0.8186657428741455
loss = 0.4644258916378021, acc = 0.8188884258270264
loss = 0.46338826417922974, acc = 0.8190013766288757
loss = 0.46245309710502625, acc = 0.8187944889068604
val acc = [0.8186644]


In [ ]:
train_df.head(10)